In [1]:
from fastai import *
from fastai.vision import *
import pandas as pd

In [2]:
labels = pd.read_csv('../input/train.csv')
def get_labels(name):
    return labels[labels['id'] == name.name]['has_cactus'].values[0]

In [3]:
data = ImageList.from_folder('../input/train/train/').split_by_rand_pct(0.01).label_from_func(get_labels).transform(get_transforms(), size=224).add_test(ItemList.from_folder('../input/test/test/')).databunch(bs=64).normalize(imagenet_stats)

In [4]:
learner = cnn_learner(data=data, base_arch=models.densenet201, model_dir='../../../../models/', metrics=accuracy)

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /tmp/.torch/models/densenet201-c1103571.pth
81131730it [00:00, 81596880.21it/s]


In [5]:
learner.fit_one_cycle(5, slice(5e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.080792,0.178653,0.954286,03:05
1,0.114067,0.025647,0.994286,02:49
2,0.038472,0.000308,1.000000,02:48
3,0.009997,0.000245,1.000000,02:48
4,0.003998,0.000018,1.000000,02:51


In [6]:
preds = learner.get_preds(DatasetType.Test)

In [7]:
predictions = preds[0].argmax(dim=1)
names = [item.name for item in data.test_ds.x.items]
pd.DataFrame({'id': names, 'has_cactus': predictions}).to_csv('submission.csv', index=False)